In [1]:
#gcp에서 데이터 복사해오기
!gsutil cp gs://chem_dsrc/P0_Toxcast/Data/toxcast_assay_list_기전.csv ./

Copying gs://chem_dsrc/P0_Toxcast/Data/toxcast_assay_list_기전.csv...
- [1 files][ 57.2 KiB/ 57.2 KiB]                                                
Operation completed over 1 objects/57.2 KiB.                                     


In [2]:
#assay에 대한 정보가 담긴 csv 읽어오기
import pandas as pd 
toxcast_assay_list_기전 = pd.read_csv("toxcast_assay_list_기전.csv")
toxcast_assay_list_기전

,Tox mechanism,AEID,Assay Name,Intended target,Intended target family,Intended target subfamily,Number of chemicals,Active chemicals,Active chemicals (%),Inactive chemicals,Model,Assay format,Time,Function,Detection,Positive Chemical
0,,806,TOX21_AhR_LUC_Agonist,AHR,dna binding,basic helix-loop-helix protein,7932,809,10.2,7123,HepG2,microplate: 1536-well plate,24.0,reporter gene,Luminescence,Omeprazole
1,AHR-cyp,767,TOX21_Aromatase_Inhibition,CYP19A1,cyp,steroidogenesis-related,7197,1169,16.2,6028,MCF-7,microplate: 1536-well plate,24.0,reporter gene,Luminescence,Letrozole
2,AHR-cyp,1131,TOX21_VDR_BLA_agonist_ratio,CYP24A1,cyp,xenobiotic metabolism,7197,70,1.0,7127,HEK293T,microplate: 1536-well plate,24.0,reporter gene,Fluorescence,Calcitriol
3,AHR-cyp,1132,TOX21_VDR_BLA_antagonist_ratio,CYP24A1,cyp,xenobiotic metabolism,7197,314,4.4,6883,HEK293T,microplate: 1536-well plate,24.0,reporter gene,Fluorescence,NaN
4,AHR-cyp,63,ATG_Ahr_CIS_up,AHR,dna binding,basic helix-loop-helix protein,3420,448,13.1,2972,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,6-formylindolo carbazole
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,PPAR,102,ATG_PPRE_CIS_up,PPARA|PPARD|PPARG,nuclear receptor,non-steroidal,3420,536,15.7,2884,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,Rosiglitazone
335,PPAR,133,ATG_PPARd_TRANS_up,PPARD,nuclear receptor,non-steroidal,3420,72,2.1,3348,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,GW7647
336,PPAR,134,ATG_PPARg_TRANS_up,PPARG,nuclear receptor,non-steroidal,3420,939,27.5,2481,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,GW7647
337,PPAR,719,NVS_NR_hPPARg,PPARG,nuclear receptor,non-steroidal,782,180,23.0,602,cell-free,microplate: 384-well plate,2.0,binding,Fluorescence,Ciglitazone


In [4]:
#active비율로 정렬하여 assay별 데이터active비율로 정렬하여 assay별 데이터균형도 확인하기
toxcast_assay_list_기전.sort_values(by = 'Active chemicals (%)',ascending=False )


,Tox mechanism,AEID,Assay Name,Intended target,Intended target family,Intended target subfamily,Number of chemicals,Active chemicals,Active chemicals (%),Inactive chemicals,Model,Assay format,Time,Function,Detection,Positive Chemical
7,AHR-cyp,337,NVS_ADME_hCYP2C9,CYP2C9,cyp,xenobiotic metabolism,225,187,83.1,38,cell-free,microplate: 96-well plate,0.75,enzymatic activity,Fluorescence,Sulfaphenazole
222,Metabolism,1508,NCCT_TPO_AUR_dn,Tpo,oxidoreductase,peroxidase,374,308,82.4,66,tissue-based cell-free,microplate: 384-well plate,0.50,binding,Fluorescence,methimazole (MMI)
5,AHR-cyp,333,NVS_ADME_hCYP2C19,CYP2C19,cyp,xenobiotic metabolism,381,305,80.1,76,cell-free,microplate: 96-well plate,1.00,enzymatic activity,Fluorescence,Tranylcypromine
256,Neuro,732,NVS_TR_hDAT,SLC6A3,transporter,neurotransmitter transporter,245,195,79.6,50,cell-free,microplate: 96-well plate,1.00,binding,Radiometry,GBR-12909
9,AHR-cyp,323,NVS_ADME_hCYP1A2,CYP1A2,cyp,xenobiotic metabolism,172,133,77.3,39,cell-free,microplate: 96-well plate,0.50,enzymatic activity,Fluorescence,Furafylline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Inflammation,94,ATG_NF_kB_CIS_up,NFKB1,dna binding,NF-kappa B,3420,22,0.6,3398,HepG2,microplate: 24-well plate,24.00,reporter gene,Fluorescence,NaN
210,Metabolism,91,ATG_MRE_CIS_up,MTF1,dna binding,zinc finger,3420,22,0.6,3398,HepG2,microplate: 24-well plate,24.00,reporter gene,Fluorescence,NaN
36,Cancer,92,ATG_Myb_CIS_up,MYB,dna binding,MYB proteins,3420,22,0.6,3398,HepG2,microplate: 24-well plate,24.00,reporter gene,Fluorescence,4-Hydroxynonenal
37,Cancer,93,ATG_Myc_CIS_up,MYC,dna binding,basic helix-loop-helix leucine zipper,3420,22,0.6,3398,HepG2,microplate: 24-well plate,24.00,reporter gene,Fluorescence,NaN


In [5]:
toxcast_assay_list_기전.sort_values(by = 'Active chemicals (%)',ascending=False ).to_csv("toxcast_assay_list_기전_sorted.csv")

#총 339개의 assay

In [6]:
toxcast_assay_list_기전['Assay Name']

0               TOX21_AhR_LUC_Agonist
1          TOX21_Aromatase_Inhibition
2         TOX21_VDR_BLA_agonist_ratio
3      TOX21_VDR_BLA_antagonist_ratio
4                      ATG_Ahr_CIS_up
                    ...              
334                   ATG_PPRE_CIS_up
335                ATG_PPARd_TRANS_up
336                ATG_PPARg_TRANS_up
337                     NVS_NR_hPPARg
338                     NVS_NR_hPPARa
Name: Assay Name, Length: 339, dtype: object

# 최신 Toxcast데이터로부터 전처리 한 949개 assay에 대한 데이터 불러오기


---



*   toxcast 홈페이지 (https://www.epa.gov/chemical-research/exploring-toxcast-data-downloadable-data) 에 "ToxCast & Tox21 Summary Files for invitroDBv3.3"파일 
*   1473개 assay중 타켓유전자가 기록되어있는 949개 assay에 한 데이터 가져옴 --> 기록되어있지 않은 524개 assay중 세포독성을 보는 assay는 100개, 나머지는 타켓이 없진 않을텐데 적혀있지는 않음(공식적으로는 타켓을 모르는 것)


*   949개에 대해선 아직 독성기전 분류를 해놓지 않아서 독성예측 결과를 해석하기는 어려운 상태 
*   따라서 예전에 Deepchem 데이터에서 타켓이 있는 339개 assay에 관해 정리해온 것을 바탕으로 339개 assay에 포함된 assay만가지고 작업할 것임


*   DeepChem_339개중 toxcast_949개에 없는 assay는 4개 따라서 335개 assay를 가지고 독성예측 모델 학습 가능









In [7]:
!gsutil cp gs://chem_dsrc/P0_Toxcast/Data/Toxcast_Assay.csv ./

Copying gs://chem_dsrc/P0_Toxcast/Data/Toxcast_Assay.csv...
- [1 files][ 10.2 MiB/ 10.2 MiB]                                                
Operation completed over 1 objects/10.2 MiB.                                     


In [8]:
assays_data = pd.read_csv('Toxcast_Assay.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,26

In [9]:
assays_data

,Source,Name,AEID,Target family,Target subfamily,Target gene,Species,Tissue,Cell,Target type,Biological process,Time(h),Chemicals,Active,Active%,100-00-5,1000-05-1,10001-13-5,100-01-6,100-02-7,10004-44-1,100-06-1,100-10-7,100-11-8,100-14-1,100-15-2,10016-20-3,100-18-5,100-19-6,100-20-9,100-21-0,100-22-1,10022-28-3,10022-31-8,10023-54-8,10024-93-8,100-25-4,10025-74-8,10025-82-8,10025-87-3,...,NOCAS_903857,NOCAS_903858,NOCAS_903859,NOCAS_903860,NOCAS_903861,NOCAS_903862,NOCAS_903863,NOCAS_903864,NOCAS_903865,NOCAS_903866,NOCAS_903867,NOCAS_903868,NOCAS_903869,NOCAS_903870,NOCAS_903871,NOCAS_903872,NOCAS_903873,NOCAS_903874,NOCAS_903875,NOCAS_903876,NOCAS_903877,NOCAS_903878,NOCAS_903879,NOCAS_903880,NOCAS_903881,NOCAS_903882,NOCAS_903883,NOCAS_903884,NOCAS_903885,NOCAS_903886,NOCAS_903887,NOCAS_903888,NOCAS_903889,NOCAS_903890,NOCAS_903891,NOCAS_903892,NOCAS_903893,NOCAS_903894,NOCAS_903895,NOCAS_903896
0,TOX21,TOX21_Aromatase_Inhibition,767,cyp,steroidogenesis-related,CYP19A1,human,breast,MCF-7,protein,regulation of transcription factor activity,24.0,8305,167,2.0,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,N,N,N,NaN,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TOX21,TOX21_AhR_LUC_Agonist,806,DNA binding,basic helix-loop-helix protein,AHR,human,liver,HepG2,protein,regulation of transcription factor activity,24.0,8305,326,3.9,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,N,N,N,NaN,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TOX21,TOX21_p53_BLA_p1_ratio,1116,DNA binding,tumor suppressor,TP53,human,intestinal,HCT116,pathway,regulation of transcription factor activity,24.0,8305,106,1.3,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,N,N,N,NaN,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TOX21,TOX21_p53_BLA_p2_ratio,1317,DNA binding,tumor suppressor,TP53,human,intestinal,HCT116,pathway,regulation of transcription factor activity,24.0,8305,236,2.8,N,N,N,N,N,N,N,N,0.32833439,N,N,N,N,N,N,N,N,NaN,N,N,N,NaN,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TOX21,TOX21_p53_BLA_p3_ratio,1321,DNA binding,tumor suppressor,TP53,human,intestinal,HCT116,pathway,regulation of transcription factor activity,24.0,8305,194,2.3,N,N,N,N,N,N,N,N,51.22480143,N,N,N,N,N,N,N,N,NaN,N,N,N,NaN,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,NVS,NVS_ENZ_hCHK1_Activator,418,kinase,serine/threonine kinase,CHEK1,human,NaN,NaN,protein,regulation of catalytic activity,1.0,2,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
945,NVS,NVS_ENZ_hLynB,471,kinase,non-receptor tyrosine kinase,LYN,human,NaN,NaN,protein,regulation of catalytic activity,1.0,1,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
946,NVS,NVS_ENZ_hLynB_

In [10]:
assays_data.sort_values(by = 'Active%',ascending=False ).to_csv("Toxcast_Assay_sorted.csv")

In [11]:
assays_data = assays_data.drop(['Source',
 'AEID',
 'Target family',
 'Target subfamily',
 'Target gene',
 'Species',
 'Tissue',
 'Cell',
 'Target type',
 'Biological process',
 'Time(h)',
 'Chemicals',
 'Active',
 'Active%'], axis = 1)

In [12]:
assays_data.index = assays_data.Name

기전별로 정리된 339개 assay(deepchem으로부터 얻어온것)중 Assay_ToxCast_ToxRefDB(toxcast로부터 얻어온것)에 있는 949개 assay에 없는 것 4개 --> 우선 제외하기.

In [13]:
no_a_list = []
for assay in list(toxcast_assay_list_기전['Assay Name']) : 
  if not assay in list(assays_data.Name)  :
    no_a_list.append(assay)
no_a_list

['TOX21_AR_LUC_MDAKB2_Antagonist',
 'TOX21_ERa_LUC_BG1_Agonist',
 'TOX21_ERa_LUC_BG1_Antagonist',
 'ACEA_T47D_80hr_Positive']

In [14]:
assays_data.Name

Name
TOX21_Aromatase_Inhibition    TOX21_Aromatase_Inhibition
TOX21_AhR_LUC_Agonist              TOX21_AhR_LUC_Agonist
TOX21_p53_BLA_p1_ratio            TOX21_p53_BLA_p1_ratio
TOX21_p53_BLA_p2_ratio            TOX21_p53_BLA_p2_ratio
TOX21_p53_BLA_p3_ratio            TOX21_p53_BLA_p3_ratio
                                         ...            
NVS_ENZ_hCHK1_Activator          NVS_ENZ_hCHK1_Activator
NVS_ENZ_hLynB                              NVS_ENZ_hLynB
NVS_ENZ_hLynB_Activator          NVS_ENZ_hLynB_Activator
NVS_NR_mERa                                  NVS_NR_mERa
NVS_ENZ_hPPP3CA                          NVS_ENZ_hPPP3CA
Name: Name, Length: 949, dtype: object

In [15]:
assay_list_339 = list(toxcast_assay_list_기전['Assay Name'])
for i in no_a_list : 
  assay_list_339.remove(i)

In [16]:
len(assay_list_339)

335

# 전체 949개 assay에 대한 데이터에서 해당 335개 assay만 추출하기

In [17]:
assays_data_335 = assays_data.loc[assay_list_339,].drop('Name', axis =1) 

In [18]:
assays_data_335 = assays_data_335.transpose()

In [20]:
assays_data_335

Name,TOX21_AhR_LUC_Agonist,TOX21_Aromatase_Inhibition,TOX21_VDR_BLA_agonist_ratio,TOX21_VDR_BLA_antagonist_ratio,ATG_Ahr_CIS_up,NVS_ADME_hCYP2C19,NVS_ADME_hCYP19A1,NVS_ADME_hCYP2C9,NVS_ADME_hCYP1A1,NVS_ADME_hCYP1A2,NVS_ADME_hCYP3A4,NVS_ADME_hCYP2B6,NVS_ADME_hCYP2D6,NVS_ADME_hCYP2A6,NVS_ADME_hCYP4F12,NVS_ADME_rCYP2C12,TOX21_AR_BLA_Agonist_ratio,TOX21_AR_BLA_Antagonist_ratio,TOX21_AR_LUC_MDAKB2_Agonist,ATG_AR_TRANS_up,OT_AR_ARELUC_AG_1440,OT_AR_ARSRC1_0480,OT_AR_ARSRC1_0960,NVS_NR_rAR,NVS_NR_hAR,TOX21_p53_BLA_p1_ratio,TOX21_p53_BLA_p2_ratio,TOX21_p53_BLA_p3_ratio,TOX21_p53_BLA_p4_ratio,TOX21_p53_BLA_p5_ratio,TOX21_HSE_BLA_agonist_ratio,TOX21_ELG1_LUC_Agonist,ATG_p53_CIS_up,ATG_E2F_CIS_up,ATG_HSE_CIS_up,ATG_Myb_CIS_up,ATG_Myc_CIS_up,BSK_BE3C_uPA_down,BSK_KF3CT_uPA_down,APR_HepG2_p53Act_24h_up,...,NVS_GPCR_rAdra1B,NVS_GPCR_r5HT_NonSelective,NVS_GPCR_r5HT1_NonSelective,NVS_GPCR_hAdra2A,NVS_GPCR_hH1,NVS_ENZ_hPTPN11,NVS_GPCR_hAdoRA2a,NVS_GPCR_hAdrb3,NVS_GPCR_rNK3,NVS_LGIC_rNNR_BungSens,NVS_GPCR_hM1,NVS_ENZ_hPDE10,NVS_ENZ_hPTPN12,NVS_GPCR_rAdra2_NonSelective,NVS_GPCR_hAdrb2,NVS_GPCR_hDRD4.4,NVS_TR_rNET,NVS_TR_hAdoT,NVS_ENZ_hPTPN13,NVS_GPCR_rSST,NVS_GPCR_hLTB4_BLT1,NVS_GPCR_hAT1,NVS_LGIC_bGABARa1,NVS_LGIC_h5HT3,NVS_GPCR_rAdrb_NonSelective,NVS_ENZ_hTrkA,NVS_LGIC_rGABAR_NonSelective,NVS_GPCR_bDR_NonSelective,NVS_GPCR_rV1,NVS_LGIC_hNNR_NBungSens,TOX21_PPARg_BLA_Agonist_ratio,TOX21_PPARd_BLA_agonist_ratio,TOX21_PPARd_BLA_antagonist_ratio,TOX21_PPARg_BLA_antagonist_ratio,ATG_PPARa_TRANS_up,ATG_PPRE_CIS_up,ATG_PPARd_TRANS_up,ATG_PPARg_TRANS_up,NVS_NR_hPPARg,NVS_NR_hPPARa
100-00-5,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,NaN
1000-05-1,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,NaN,NaN
10001-13-5,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN
100-01-6,N,N,N,N,N,6.449239794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.086069804,NaN,NaN,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,NaN,NaN
100-02-7,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,N,N,N,NaN,NaN,0.000103442,N,0.000122936,N,N,N,N,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,...,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NOCAS_903892,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,NaN,NaN
NOCAS_903893,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,NaN,NaN
NOCAS_903894,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

#"toxcast_assay_list_기전"파일에 미리 기록되어있던  active 비율과 새로 만든 assay_data_335의 active 비율 같은지 확인하기




In [19]:
assays_data_335.columns

Index(['TOX21_AhR_LUC_Agonist', 'TOX21_Aromatase_Inhibition',
       'TOX21_VDR_BLA_agonist_ratio', 'TOX21_VDR_BLA_antagonist_ratio',
       'ATG_Ahr_CIS_up', 'NVS_ADME_hCYP2C19', 'NVS_ADME_hCYP19A1',
       'NVS_ADME_hCYP2C9', 'NVS_ADME_hCYP1A1', 'NVS_ADME_hCYP1A2',
       ...
       'TOX21_PPARg_BLA_Agonist_ratio', 'TOX21_PPARd_BLA_agonist_ratio',
       'TOX21_PPARd_BLA_antagonist_ratio', 'TOX21_PPARg_BLA_antagonist_ratio',
       'ATG_PPARa_TRANS_up', 'ATG_PPRE_CIS_up', 'ATG_PPARd_TRANS_up',
       'ATG_PPARg_TRANS_up', 'NVS_NR_hPPARg', 'NVS_NR_hPPARa'],
      dtype='object', name='Name', length=335)

In [21]:
dict_assay_noc = {}
for index, assay in enumerate(toxcast_assay_list_기전['Assay Name']) : 
  dict_assay_noc[assay] =  toxcast_assay_list_기전['Number of chemicals'][index]

dict_assay_ac = {}
for index, assay in enumerate(toxcast_assay_list_기전['Assay Name']) : 
  dict_assay_ac[assay] =  toxcast_assay_list_기전['Active chemicals'][index]

In [22]:
new_dict_assay_noc = {}
new_dict_assay_ac = {}

for i in assays_data_335.columns : 
  chemicals = assays_data_335[i].dropna()
  noc = len(chemicals)
  try : 
    ac  = noc - chemicals.value_counts()['N']
  except : 
    ac = 0
  new_dict_assay_noc[i] = noc
  new_dict_assay_ac[i] = ac


#기록된 previous 데이터 와 최근 데이터사이에 gap이 큼

Assay_ToxCast_ToxRefDB에 있는걸로 다시 number of chemicals, active 수 다시 세 보았는데 기록되어있는것(toxcast_assay_list_기전 파일)이랑 차이가 많이 남 

assay당 전체 chemcial수는 몇십개에서 몇백개정도 더 많은데 오히려 active chemcial수는 많게는 1000개정도 더 적음

저번 것 이후에 새로운 데이터를 포함한 것이라 물질 수는 늘어남. 

그런데 z-score를 고려해서 세포독성 나타나는 농도에서 활성을 보인 것은 비활성으로 해서 활성물질 수는 줄어듬

전체 toxcast asssay중에 34개가 세포독성을 보는 assay

거기서 나오는 ac50값을 표준화 시켜서 0으로 놓고 z-space를 만든다

 z-score가 3보다 큰 것만 active로 함.

 

epa toxcast에서 assay랑 chemcial 별로 z-score matrix를 제공 

화학물질이 assay에 대해서 실험을 했을때 세포독성이 먼저 나타나 버리면 타겟 유전자자가 특이적으로 발생하는지 아닌지 모름. 세포독성에 의해서 연쇄적으로 나타난 것일 수 있기 때문 따라서 특이적으로 나타나는것을 보기 위해서 z-score가 3이상인 것만 활성으로 봄. 

논문에서 z-score가 3 이하이면  세포독성이 나타나는 농도인 것임 따라서 타켓 유전자 assay에 대한 chemical의 활성을 알아볼때에  z-score matrix에서 3 이하인 것은 활성이 아니라고 한 것이다. 

특정 타겟 유전자 발현은 물질을 노출했을때. 유전자가 발현한다는 것을 나타낸다. 실험하다가 고농도로하면 세포가 죽어버릴 수도 있으니까, 수많은 유전자가 발현할 수 있다. 

In [24]:
compare_orignal_zscore = pd.DataFrame(index=assays_data_335.columns, columns=['orignal Number of Chemcials','zscore processed Number of Chemcials',
                                                       'orignal Number of acitves','zscore processed Number of actives',
                                                       'orignal Number of acitves(%)','zscore processed Number of actives(%)',]
                                      )

In [25]:
compare_orignal_zscore

,orignal Number of Chemcials,zscore processed Number of Chemcials,orignal Number of acitves,zscore processed Number of actives,orignal Number of acitves(%),zscore processed Number of actives(%)
Name,,,,,,
TOX21_AhR_LUC_Agonist,NaN,NaN,NaN,NaN,NaN,NaN
TOX21_Aromatase_Inhibition,NaN,NaN,NaN,NaN,NaN,NaN
TOX21_VDR_BLA_agonist_ratio,NaN,NaN,NaN,NaN,NaN,NaN
TOX21_VDR_BLA_antagonist_ratio,NaN,NaN,NaN,NaN,NaN,NaN
ATG_Ahr_CIS_up,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
ATG_PPRE_CIS_up,NaN,NaN,NaN,NaN,NaN,NaN
ATG_PPARd_TRANS_up,NaN,NaN,NaN,NaN,NaN,NaN
ATG_PPARg_TRANS_up,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
for i in assays_data_335 : 
  compare_orignal_zscore.loc[i,'orignal Number of Chemcials' ] = dict_assay_noc[i]
  compare_orignal_zscore.loc[i,'zscore processed Number of Chemcials' ] = new_dict_assay_noc[i]

  compare_orignal_zscore.loc[i,'orignal Number of acitves' ] = dict_assay_ac[i]
  compare_orignal_zscore.loc[i,'zscore processed Number of actives' ] = new_dict_assay_ac[i]

  compare_orignal_zscore.loc[i,'orignal Number of acitves(%)' ] = round( 100 * dict_assay_ac[i] / dict_assay_noc[i] ,2)
  try  : 
    compare_orignal_zscore.loc[i,'zscore processed Number of actives(%)'] = round( 100 * new_dict_assay_ac[i] / new_dict_assay_noc[i] ,2)
  except : 
    compare_orignal_zscore.loc[i,'zscore processed Number of actives(%)'] = 0

In [27]:
compare_orignal_zscore.sort_values(by = 'zscore processed Number of actives(%)',ascending=False ).to_csv("compare_orignal_zscore.csv")

In [28]:
compare_orignal_zscore = compare_orignal_zscore.sort_values(by = 'zscore processed Number of actives(%)',ascending=False )

In [29]:
compare_orignal_zscore.loc[['NVS_ADME_hCYP1A2'  , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn',
                        'TOX21_AhR_LUC_Agonist', 'TOX21_Aromatase_Inhibition', 'TOX21_VDR_BLA_agonist_ratio', 'TOX21_VDR_BLA_antagonist_ratio', 'TOX21_AR_BLA_Agonist_ratio', 'TOX21_AR_BLA_Antagonist_ratio', 'TOX21_AR_LUC_MDAKB2_Agonist', 'TOX21_p53_BLA_p1_ratio', 'TOX21_p53_BLA_p2_ratio', 'TOX21_p53_BLA_p3_ratio', 'TOX21_p53_BLA_p4_ratio', 'TOX21_p53_BLA_p5_ratio', 'TOX21_HSE_BLA_agonist_ratio', 'TOX21_ELG1_LUC_Agonist', 'TOX21_ERa_BLA_Agonist_ratio', 'TOX21_ERa_BLA_Antagonist_ratio', 'TOX21_ESRE_BLA_ratio', 'TOX21_FXR_BLA_agonist_ratio', 'TOX21_FXR_BLA_antagonist_ratio', 'TOX21_GR_BLA_Agonist_ratio', 'TOX21_GR_BLA_Antagonist_ratio', 'TOX21_NFkB_BLA_agonist_ratio', 'TOX21_ARE_BLA_agonist_ratio', 'TOX21_TR_LUC_GH3_Agonist', 'TOX21_TR_LUC_GH3_Antagonist', 'TOX21_PPARg_BLA_Agonist_ratio', 'TOX21_PPARd_BLA_agonist_ratio', 'TOX21_PPARd_BLA_antagonist_ratio', 'TOX21_PPARg_BLA_antagonist_ratio']
                       ].sort_values(by = 'zscore processed Number of actives(%)',ascending=False ).to_csv("compare_orignal_zscore_3_29.csv")

In [30]:
compare_orignal_zscore

,orignal Number of Chemcials,zscore processed Number of Chemcials,orignal Number of acitves,zscore processed Number of actives,orignal Number of acitves(%),zscore processed Number of actives(%)
Name,,,,,,
NVS_ADME_hCYP2C19,381,459,305,301,80.05,65.58
NVS_ADME_hCYP2D6,117,162,80,104,68.38,64.2
NVS_ADME_hCYP2C9,225,266,187,160,83.11,60.15
NVS_ADME_hCYP1A2,172,175,133,102,77.33,58.29
NVS_ADME_hCYP2B6,138,140,103,81,74.64,57.86
...,...,...,...,...,...,...
ATG_AP_2_CIS_up,3420,3849,44,7,1.29,0.18
ATG_ERRg_TRANS_up,3420,3849,83,4,2.43,0.1
ATG_HNF4a_TRANS_up,3420,3849,50,3,1.46,0.08


In [31]:
for i in assays_data_335 : 
  print("number of chemicals & active " , i)
  print("previous : ",dict_assay_noc[i], "now : ", new_dict_assay_noc[i])
  print("previous : ",dict_assay_ac[i], "now : ", new_dict_assay_ac[i])
  print()

number of chemicals & active  TOX21_AhR_LUC_Agonist
previous :  7932 now :  8305
previous :  809 now :  326

number of chemicals & active  TOX21_Aromatase_Inhibition
previous :  7197 now :  8305
previous :  1169 now :  167

number of chemicals & active  TOX21_VDR_BLA_agonist_ratio
previous :  7197 now :  7521
previous :  70 now :  17

number of chemicals & active  TOX21_VDR_BLA_antagonist_ratio
previous :  7197 now :  7521
previous :  314 now :  33

number of chemicals & active  ATG_Ahr_CIS_up
previous :  3420 now :  3849
previous :  448 now :  354

number of chemicals & active  NVS_ADME_hCYP2C19
previous :  381 now :  459
previous :  305 now :  301

number of chemicals & active  NVS_ADME_hCYP19A1
previous :  239 now :  247
previous :  71 now :  64

number of chemicals & active  NVS_ADME_hCYP2C9
previous :  225 now :  266
previous :  187 now :  160

number of chemicals & active  NVS_ADME_hCYP1A1
previous :  175 now :  180
previous :  70 now :  58

number of chemicals & active  NVS_ADME

In [32]:
 chemicals = assays_data_335[i].dropna()
 len(chemicals)

396

In [33]:
chemicals.value_counts()['N']

358

In [34]:
toxcast_assay_list_기전

,Tox mechanism,AEID,Assay Name,Intended target,Intended target family,Intended target subfamily,Number of chemicals,Active chemicals,Active chemicals (%),Inactive chemicals,Model,Assay format,Time,Function,Detection,Positive Chemical
0,,806,TOX21_AhR_LUC_Agonist,AHR,dna binding,basic helix-loop-helix protein,7932,809,10.2,7123,HepG2,microplate: 1536-well plate,24.0,reporter gene,Luminescence,Omeprazole
1,AHR-cyp,767,TOX21_Aromatase_Inhibition,CYP19A1,cyp,steroidogenesis-related,7197,1169,16.2,6028,MCF-7,microplate: 1536-well plate,24.0,reporter gene,Luminescence,Letrozole
2,AHR-cyp,1131,TOX21_VDR_BLA_agonist_ratio,CYP24A1,cyp,xenobiotic metabolism,7197,70,1.0,7127,HEK293T,microplate: 1536-well plate,24.0,reporter gene,Fluorescence,Calcitriol
3,AHR-cyp,1132,TOX21_VDR_BLA_antagonist_ratio,CYP24A1,cyp,xenobiotic metabolism,7197,314,4.4,6883,HEK293T,microplate: 1536-well plate,24.0,reporter gene,Fluorescence,NaN
4,AHR-cyp,63,ATG_Ahr_CIS_up,AHR,dna binding,basic helix-loop-helix protein,3420,448,13.1,2972,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,6-formylindolo carbazole
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,PPAR,102,ATG_PPRE_CIS_up,PPARA|PPARD|PPARG,nuclear receptor,non-steroidal,3420,536,15.7,2884,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,Rosiglitazone
335,PPAR,133,ATG_PPARd_TRANS_up,PPARD,nuclear receptor,non-steroidal,3420,72,2.1,3348,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,GW7647
336,PPAR,134,ATG_PPARg_TRANS_up,PPARG,nuclear receptor,non-steroidal,3420,939,27.5,2481,HepG2,microplate: 24-well plate,24.0,reporter gene,Fluorescence,GW7647
337,PPAR,719,NVS_NR_hPPARg,PPARG,nuclear receptor,non-steroidal,782,180,23.0,602,cell-free,microplate: 384-well plate,2.0,binding,Fluorescence,Ciglitazone
